In [10]:
import torch
import numpy as np
import random
import torch.nn as nn
from torch.autograd import Variable

# from Model import MMINet
# from Data import get_dataloader


In [11]:
# %% Preprocess data into a data loader
def get_dataloader(data_B, data_F, data_T, norm, n_init=16):
    """Get a test dataloader 

    Parameters
    ---------
    data_B/data_F/data_T : array 
         B/F/T data
    norm : list 
         B/F/T normalization data
    n_init : int
         Additional points for computing the history magnetization
    """

    # Data pre-process
    # 1. Down-sample to 128 points
    seq_length = 128
    cols = range(0, 1024, int(1024 / seq_length))
    data_B = data_B[:, cols]

    # 2. Add extra points for initial magnetization calculation
    data_length = seq_length + n_init
    data_B = np.hstack((data_B, data_B[:, :n_init]))

    # 3. Format data into tensors
    B = torch.from_numpy(data_B).view(-1, data_length, 1).float()
    F = torch.log10(torch.from_numpy(data_F).view(-1, 1).float())
    T = torch.from_numpy(data_T).view(-1, 1).float()

    # 4. Data Normalization
    in_B = (B - norm[0][0]) / norm[0][1]
    in_T = (T - norm[3][0]) / norm[3][1]
    in_F = (F - norm[2][0]) / norm[2][1]

    # 5. Extra features
    dB = torch.diff(B, dim=1)
    dB = torch.cat((dB[:, 0:1], dB), dim=1)
    dB_dt = dB * (seq_length * F.view(-1, 1, 1))

    in_dB = torch.diff(B, dim=1)  # Flux density change
    in_dB = torch.cat((in_dB[:, 0:1], in_dB), dim=1)

    in_dB_dt = (dB_dt - norm[4][0]) / norm[4][1]  # Flux density change rate

    max_B, _ = torch.max(in_B, dim=1)
    min_B, _ = torch.min(in_B, dim=1)

    s0 = get_operator_init(in_B[:, 0] - in_dB[:, 0], in_dB, max_B,
                           min_B)  # Operator inital state

    # 6. Create dataloader to speed up data processing
    test_dataset = torch.utils.data.TensorDataset(
        torch.cat((in_B, in_dB, in_dB_dt), dim=2),
        torch.cat((in_F, in_T), dim=1), s0)
    kwargs = {
        'num_workers': 4,
        'batch_size': 128,
        'pin_memory': True,
        'pin_memory_device': "cuda" if torch.cuda.is_available() else "cpu",
        'drop_last': False
    }
    test_loader = torch.utils.data.DataLoader(test_dataset, **kwargs)

    return test_loader

In [12]:
# %% Predict the operator state at t0
def get_operator_init(B1, dB, Bmax, Bmin, max_out_H=5, operator_size=30):
    """Compute the inital state of hysteresis operators

    Parameters
    ---------
    B1 : torch_like (batch)
         Stop operator excitation at t1
    dB : torch_like (batch, data_length)
         Flux density changes at each t
    Bmax/Bmin : torch_like (batch)
         Max/Min flux density of each cycle 
    """
    # 1. Parameter setting
    s0 = torch.zeros((dB.shape[0], operator_size))  # Allocate cache for s0
    operator_thre = torch.from_numpy(
        np.linspace(max_out_H / operator_size, max_out_H,
                    operator_size)).view(1,
                                         -1)  # hysteresis operators' threshold

    # 2. Iterate each excitation for the operator inital state computation
    for i in range(dB.shape[0]):
        for j in range(operator_size):
            r = operator_thre[0, j]
            if (Bmax[i] >= r) or (Bmin[i] <= -r):
                if dB[i, 0] >= 0:
                    if B1[i] > Bmin[i] + 2 * r:
                        s0[i, j] = r
                    else:
                        s0[i, j] = B1[i] - (r + Bmin[i])
                else:
                    if B1[i] < Bmax[i] - 2 * r:
                        s0[i, j] = -r
                    else:
                        s0[i, j] = B1[i] + (r - Bmax[i])

    return s0

In [13]:
# %% Preprocess data into a data loader
def get_dataloader(data_B, data_F, data_T, norm, n_init=16):
    """Get a test dataloader 

    Parameters
    ---------
    data_B/data_F/data_T : array 
         B/F/T data
    norm : list 
         B/F/T normalization data
    n_init : int
         Additional points for computing the history magnetization
    """

    # Data pre-process
    # 1. Down-sample to 128 points
    seq_length = 128
    cols = range(0, 1024, int(1024 / seq_length))
    data_B = data_B[:, cols]

    # 2. Add extra points for initial magnetization calculation
    data_length = seq_length + n_init
    data_B = np.hstack((data_B, data_B[:, :n_init]))

    # 3. Format data into tensors
    B = torch.from_numpy(data_B).view(-1, data_length, 1).float()
    F = torch.log10(torch.from_numpy(data_F).view(-1, 1).float())
    T = torch.from_numpy(data_T).view(-1, 1).float()

    # 4. Data Normalization
    in_B = (B - norm[0][0]) / norm[0][1]

    in_T = (T - norm[3][0]) / norm[3][1]
    in_F = (F - norm[2][0]) / norm[2][1]

    # 5. Extra features
    dB = torch.diff(B, dim=1)
    dB = torch.cat((dB[:, 0:1], dB), dim=1)
    dB_dt = dB * (seq_length * F.view(-1, 1, 1))

    in_dB = torch.diff(B, dim=1)  # Flux density change
    in_dB = torch.cat((in_dB[:, 0:1], in_dB), dim=1)

    in_dB_dt = (dB_dt - norm[4][0]) / norm[4][1]  # Flux density change rate

    max_B, _ = torch.max(in_B, dim=1)
    min_B, _ = torch.min(in_B, dim=1)

    s0 = get_operator_init(in_B[:, 0] - in_dB[:, 0], in_dB, max_B,
                           min_B)  # Operator inital state

    # 6. Create dataloader to speed up data processing
    test_dataset = torch.utils.data.TensorDataset(
        torch.cat((in_B, in_dB, in_dB_dt), dim=2),
        torch.cat((in_F, in_T), dim=1), s0)
    kwargs = {
        'num_workers': 4,
        'batch_size': 128,
        'pin_memory': True,
        'pin_memory_device': "cuda" if torch.cuda.is_available() else "cpu",
        'drop_last': False
    }
    test_loader = torch.utils.data.DataLoader(test_dataset, **kwargs)

    return test_loader


In [14]:
# %% Predict the operator state at t0
def get_operator_init(B1, dB, Bmax, Bmin, max_out_H=5, operator_size=30):
    """Compute the inital state of hysteresis operators

    Parameters
    ---------
    B1 : torch_like (batch)
         Stop operator excitation at t1
    dB : torch_like (batch, data_length)
         Flux density changes at each t
    Bmax/Bmin : torch_like (batch)
         Max/Min flux density of each cycle 
    """
    # 1. Parameter setting
    s0 = torch.zeros((dB.shape[0], operator_size))  # Allocate cache for s0
    operator_thre = torch.from_numpy(
        np.linspace(max_out_H / operator_size, max_out_H,
                    operator_size)).view(1,
                                         -1)  # hysteresis operators' threshold

    # 2. Iterate each excitation for the operator inital state computation
    for i in range(dB.shape[0]):
        for j in range(operator_size):
            r = operator_thre[0, j]
            if (Bmax[i] >= r) or (Bmin[i] <= -r):
                if dB[i, 0] >= 0:
                    if B1[i] > Bmin[i] + 2 * r:
                        s0[i, j] = r
                    else:
                        s0[i, j] = B1[i] - (r + Bmin[i])
                else:
                    if B1[i] < Bmax[i] - 2 * r:
                        s0[i, j] = -r
                    else:
                        s0[i, j] = B1[i] + (r - Bmax[i])

    return s0

In [15]:
# Material normalization data (1.B 2.H 3.F 4.T 5.dB/dt 6.Pv)
normsDict = {
    "Material A": [[-4.02296069e-19, 6.42790612e-02],
                   [1.15118525e-01, 1.22041107e+01],
                   [5.16368866e+00, 2.68540382e-01],
                   [5.52569885e+01, 2.61055470e+01],
                   [2.42224485e-01, 2.37511802e+00],
                   [4.94751596e+00, 8.27844262e-01]],
    "Material B": [[6.75135623e-20, 6.27030179e-02],
                   [3.95575739e-02, 7.62486081e+00],
                   [5.26432657e+00, 2.88519919e-01],
                   [5.80945930e+01, 2.40673885e+01],
                   [2.72521585e-01, 2.46433449e+00],
                   [5.05083704e+00, 7.10303366e-01]],
    "Material C": [[-7.61633305e-19, 7.95720905e-02],
                   [1.11319124e-01, 1.30629103e+01],
                   [5.18559408e+00, 2.68714815e-01],
                   [5.84123573e+01, 2.40717468e+01],
                   [3.26634765e-01, 3.03949690e+00],
                   [4.74633312e+00, 8.05532336e-01]],
    "Material D": [[-3.82835526e-18, 8.10498434e-02],
                   [-1.14488902e-02, 2.83868927e+01],
                   [5.25141287e+00, 2.50821203e-01],
                   [6.72413788e+01, 2.59518223e+01],
                   [3.00584078e-01, 3.24369454e+00],
                   [5.01819372e+00, 8.41059685e-01]],
    "Material E": [[-4.22607249e-18, 1.28762770e-01],
                   [3.88389004e-01, 4.80431443e+01],
                   [5.18909550e+00, 2.77695119e-01],
                   [5.64505730e+01, 2.46127701e+01],
                   [6.35038793e-01, 5.19237566e+00],
                   [5.68955612e+00, 7.26979315e-01]]
}


# %% Magnetization mechansim-determined neural network
class MMINet(nn.Module):
    """
     Parameters:
      - hidden_size: number of eddy current slices (RNN neuron)
      - operator_size: number of operators
      - input_size: number of inputs (1.B 2.dB 3.dB/dt)
      - var_size: number of supplenmentary variables (1.F 2.T)        
      - output_size: number of outputs (1.H)
    """

    def __init__(self,
                 Material,
                 hidden_size=30,
                 operator_size=30,
                 input_size=3,
                 var_size=2,
                 output_size=1):
        super().__init__()
        self.input_size = input_size
        self.var_size = var_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.operator_size = operator_size
        self.norm = normsDict[Material]  # normalization data

        # Consturct the network
        self.rnn1 = StopOperatorCell(self.operator_size)
        self.dnn1 = nn.Linear(self.operator_size + 2, 1)
        self.rnn2 = EddyCell(4, self.hidden_size, output_size)
        self.dnn2 = nn.Linear(self.hidden_size, 1)

        self.rnn2_hx = None

    def forward(self, x, var, s0, n_init=16):
        """
         Parameters: 
          - x(batch,seq,input_size): Input features (1.B, 2.dB, 3.dB/dt)  
          - var(batch,var_size): Supplementary inputs (1.F 2.T)
          - s0(batch,1): Operator inital states
        """
        batch_size = x.size(0)  # Batch size
        seq_size = x.size(1)  # Ser
        self.rnn1_hx = s0

        # Initialize DNN2 input (1.B 2.dB/dt)
        x2 = torch.cat((x[:, :, 0:1], x[:, :, 2:3]), dim=2)

        for t in range(seq_size):
            # RNN1 input (dB,state)
            self.rnn1_hx = self.rnn1(x[:, t, 1:2], self.rnn1_hx)

            # DNN1 input (rnn1_hx,F,T)
            dnn1_in = torch.cat((self.rnn1_hx, var), dim=1)

            # H hysteresis prediction
            H_hyst_pred = self.dnn1(dnn1_in)

            # DNN2 input (B,dB/dt,T,F)
            rnn2_in = torch.cat((x2[:, t, :], var), dim=1)

            # Initialize second rnn state
            if t == 0:
                H_eddy_init = x[:, t, 0:1] - H_hyst_pred
                buffer = x.new_ones(x.size(0), self.hidden_size)
                self.rnn2_hx = Variable(
                    (buffer / torch.sum(self.dnn2.weight, dim=1)) *
                    H_eddy_init)

            #rnn2_in = torch.cat((rnn2_in,H_hyst_pred),dim=1)
            self.rnn2_hx = self.rnn2(rnn2_in, self.rnn2_hx)

            # H eddy prediction
            H_eddy = self.dnn2(self.rnn2_hx)

            # H total
            H_total = (H_hyst_pred + H_eddy).view(batch_size, 1,
                                                  self.output_size)
            if t == 0:
                output = H_total
            else:
                output = torch.cat((output, H_total), dim=1)

        # Compute the power loss density
        B = (x[:, n_init:, 0:1] * self.norm[0][1] + self.norm[0][0])
        H = (output[:, n_init:, :] * self.norm[1][1] + self.norm[1][0])
        Pv = torch.trapz(H, B, axis=1) * (10**(var[:, 0:1] * self.norm[2][1] +
                                               self.norm[2][0]))

        return torch.flatten(Pv)


# %% MMINN Sub-layer: Static hysteresis prediction using stop operators
class StopOperatorCell():
    """ 
      Parameters:
      - operator_size: number of operator
    """

    def __init__(self, operator_size):
        self.operator_thre = torch.from_numpy(
            np.linspace(5 / operator_size, 5, operator_size)).view(1, -1)

    def sslu(self, X):
        """ Hardsimoid-like or symmetric saturated linear unit definition

        """
        a = torch.ones_like(X)
        return torch.max(-a, torch.min(a, X))

    def __call__(self, dB, state):
        """ Update operator of each time step

        """
        r = self.operator_thre.to(dB.device)
        output = self.sslu((dB + state) / r) * r
        return output.float()


# %% MMINN subsubnetwork: Dynamic hysteresis prediction
class EddyCell(nn.Module):
    """ 
      Parameters:
      - input_size: feature size 
      - hidden_size: number of hidden units (eddy current layers)
      - output_size: number of the output
    """

    def __init__(self, input_size, hidden_size, output_size=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.x2h = nn.Linear(input_size, hidden_size, bias=False)
        self.h2h = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x, hidden=None):
        """
         Parameters:
         - x(batch,input_size): features (1.B 2.dB/dt 3.F 4.T)
         - hidden(batch,hidden_size): dynamic hysteresis effects at each eddy current layer 
        """
        hidden = self.x2h(x) + self.h2h(hidden)
        hidden = torch.sigmoid(hidden)
        return hidden


In [16]:
# Target Material
Material = "Material E"

# Select GPU as default device
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#%% Load Dataset
def load_dataset(in_file1="./Data/Testing/" + Material + "/B_Field.csv",
                 in_file2="./Data/Testing/" + Material + "/Frequency.csv",
                 in_file3="./Data/Testing/" + Material + "/Temperature.csv"):

    data_B = np.genfromtxt(in_file1, delimiter=',')  # N by 1024, in T
    data_F = np.genfromtxt(in_file2, delimiter=',')  # N by 1, in Hz
    data_T = np.genfromtxt(in_file3, delimiter=',')  # N by 1, in C

    return data_B, data_F, data_T


#%% Calculate Core Loss
def core_loss(data_B, data_F, data_T):

    #================ Wrap your model or algorithm here=======================#
    # 1.Create model isntances
    net = MMINet(Material).to(Device)

    # 2.Load specific model
    net_file = "./Model/" + Material + ".pt"
    state_dict = torch.load(net_file, map_location=Device)
    net.load_state_dict(state_dict, strict=True)

    # 3.Get dataloader
    loader = get_dataloader(data_B, data_F, data_T, net.norm)

    # 4.Validate the models
    data_P = torch.Tensor([]).to(
        Device)  # Allocate memory to store loss density

    with torch.no_grad():
        # Start model evaluation explicitly
        net.eval()

        for inputs, vars, s0 in loader:
            Pv = net(inputs.to(Device), vars.to(Device), s0.to(Device))

            data_P = torch.cat((data_P, Pv), dim=0)
        print(data_P.shape)
    #=========================================================================#
    np.savetxt("./Data/Testing/Result/Volumetric_Loss_" + Material + ".csv",
               data_P.to('cpu'),
               delimiter=',')

    print('Model inference is finished!')

    return


In [17]:
#%% Main Function for Model Inference


def main():

    # Reproducibility
    MYSEED = 1
    random.seed(MYSEED)  # Random seed
    np.random.seed(MYSEED)
    torch.manual_seed(MYSEED)  # Data Loading
    torch.backends.cudnn.deterministic = True  # Deterministic operations
    torch.backends.cudnn.benchmark = False

    # Generate dataloader
    data_B, data_F, data_T = load_dataset()

    # Predict and save file
    core_loss(data_B, data_F, data_T)


if __name__ == "__main__":
    main()

C:\Users\USER\AppData\Local\Temp\ipykernel_37492\2678163534.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(net_file, map_location=Device)


torch.Size([3738])
Model inference is finished!


In [ ]:
# net = MMINet(Material).to(Device)
# net_file="./Model/"+Material+".pt"
# state_dict = torch.load(net_file, map_location=Device)
# net.load_state_dict(state_dict,strict=True)

# for parm in net.parameters():
#     print(parm)

Parameter containing:
tensor([[ 0.0440,  0.0598,  0.0293,  0.1591, -0.0938,  0.0126,  0.0146,  0.0302,
          0.1027, -0.2004,  0.1319, -0.0054,  0.0303,  0.0011,  0.0014,  0.0328,
          0.0744,  0.0481,  0.2033,  0.1038, -0.0171, -0.0580,  0.0191, -0.0275,
         -0.0079,  0.0572,  0.1541,  0.1448, -0.1241,  0.1583,  0.0309,  0.0690]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.0628], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 1.4944e+00,  5.3623e-01,  9.9983e-01, -3.6611e-02],
        [ 9.2726e-02,  9.5585e-02,  8.0337e-02, -8.1617e-02],
        [ 1.7293e+00, -4.3700e-01, -1.6699e-01, -1.9062e+00],
        [ 2.0573e+00,  8.4392e-02,  3.0469e-02, -1.5226e-01],
        [-1.5637e-01,  3.2802e-01,  3.5908e-01, -1.5385e-01],
        [ 1.0940e-01, -9.7014e-01,  3.5369e-02, -5.3470e-02],
        [-3.6352e+00, -2.3051e-01,  7.1003e-02, -9.1589e-02],
        [-2.5176e+00, -1.0482e+00, -1.6136e-01, -6.5170e-02],
        [ 4.87

C:\Users\USER\AppData\Local\Temp\ipykernel_37492\793971333.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(net_file, map_location=Device)
